In [40]:
%matplotlib inline
from __future__ import division
import numpy as np
import math as math
from numpy.random import rand
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
import os.path
from os import path
import time

In [33]:
class Ising():
    def __init__(self, temp, n, msrmnt, animate, train, verbose):
        self.temp = temp
        self.n = n
        self.msrmnt = msrmnt
        self.animate = animate
        self.train = train
        self.verbose = verbose

    ''' Simulating the Ising model '''    
    ## monte carlo moves
    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):            
                    a = np.random.randint(0, N)
                    b = np.random.randint(0, N)
                    s =  config[a, b]
                    energy_i = -1*(np.cos(s - config[(a+1)%N,b]) + np.cos(s - config[a,(b+1)%N]) + np.cos(s - config[(a-1)%N,b]) + np.cos(s - config[a,(b-1)%N]))
                    dtheta = pow(-1,np.random.randint(0,1))*(np.pi/50)
                    spin_temp = s + dtheta
                    energy_f = -1*(np.cos(spin_temp-config[(a+1)%N,b]) + np.cos(spin_temp-config[a,(b+1)%N]) + np.cos(spin_temp-config[(a-1)%N,b]) + np.cos(spin_temp-config[a,(b-1)%N]))
                    delta_E = energy_f - energy_i
                    if np.random.uniform(0.0, 1.0) < np.exp(-1 * beta * delta_E):
                        s+= dtheta
                    config[a,b] = s%(2*np.pi)
        return config

 
    def animate_driver(self, frame, config, N, beta, f):
        config = self.mcmove(config, N, 1.0/temp)
        plot_frame = self.configPlot(f, config, N, 1);   
        return plot_frame,


    def img_naming(self):
        train = self.train
        if os.path.exists("train/") == False:
                os.mkdir("train/")
        if os.path.exists("test/") == False:
                os.mkdir("test/")
        if train == True:
            plotpath = "".join(["train/",str(self.temp),"/"])
        else:
            plotpath = "".join(["test/",str(self.temp),"/"])
        if os.path.exists(plotpath) == False:
                os.mkdir(plotpath)
        plotname = [plotpath,"IsingXY_",str(self.temp),"_"]
        plotname = "".join(plotname)
        i = 0
        while os.path.exists(f"{plotname}{i}.png"):
            i += 1
        plotname = [plotname,str(i),".png"]
        plotname = "".join(plotname)
        return plotname

    def gif_naming(self):
        if os.path.exists("animation/") == False:
                os.mkdir("animation/")
        plotpath = "".join(["animation/IsingXY_",str(self.temp),"_"])
        i = 0
        while os.path.exists(f"{plotpath}{i}.gif"):
            i += 1
        plotname = [plotpath,str(i),".gif"]
        plotname = "".join(plotname)
        print(plotname)
        return plotname

    def configPlot(self, f, config, N, n_):
        ''' This modules plts the configuration once passed to it along with time etc '''
        X, Y = np.meshgrid(range(N+1), range(N+1))
        sp =  f.add_subplot(1, 1, n_ )  
        plt.setp(sp.get_yticklabels(), visible=False)
        plt.setp(sp.get_xticklabels(), visible=False)      
        plt.gca().set_axis_off()
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
        plt.margins(0,0)
        f.patch.set_facecolor('none')
        plot_frame = plt.pcolormesh(X, Y, config, cmap=plt.cm.get_cmap('hsv'));
        return plot_frame,


    def simulate(self):   
        ''' This module simulates the Ising model'''
        verbose = self.verbose
        temp  = self.temp
        N     = self.n        # Initialse the lattice
        msrmnt = self.msrmnt
        animate = self.animate
        config = np.random.uniform(low = 0*np.pi, high = 2*np.pi, size=(N,N))
        dpi = 8*1.7325
        f = plt.figure(figsize=(N/8*1.7325/3, N/8*1.7325/3), dpi=dpi);   
        #self.configPlot(f, config, 0, N, 1);
        if animate:
            gifname = self.gif_naming()
            anim = FuncAnimation(f, self.animate_driver, fargs=(config, N, 1.0/temp, f),
                               frames=msrmnt, interval=20, blit=False)
            anim.save(gifname, writer = PillowWriter(fps = 30), dpi = dpi*5) 
            #f.savefig('test.png', transparent = True, bbox_inches = 'tight', pad_inches = 0)
            plt.close(f)
        else:
            plotname = self.img_naming()
            for i in range(msrmnt):
                if verbose == True:
                    if i % msrmnt*.05 == 0:
                        print("iteration: ",i," of ",msrmnt)
                self.mcmove(config, N, 1.0/(temp))
                if i == msrmnt:    self.configPlot(f, config, N, 1);
            print("Generated: ",plotname)
            f.savefig(plotname, transparent = True, bbox_inches = 'tight', pad_inches = 0)
            plt.close(f)
    

In [48]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

def time_parse(seconds):
    minutes = (seconds)/60
    hours = (minutes)/60
    if hours >= 1:
        hours = math.floor(hours)
        minutes = minutes % 60
        minutes = truncate(minutes, 2)
        return " ".join(["Time Elapsed:", str(hours), "hours", str(minutes),"minutes"])
    if minutes >= 1:
        minutes = math.floor(minutes)
        seconds = seconds % 60
        seconds = truncate(seconds, 2)
        return " ".join(["Time Elapsed:", str(minutes), "minutes", str(seconds),"seconds"])
    seconds = truncate(seconds, 2)
    return " ".join(["Time Elapsed:", str(seconds),"seconds"])

In [49]:
# temp_dist is the input temperature distribution
#temp_dist = np.array([0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5])
# range of temperature values - [0, temp_scale)
temp_scale = 2
# number of temperature values to generate
num_temp = 10
temp_dist = (np.random.rand(1,num_temp)*temp_scale)[0]
# n is resolution of image (nxn)
N = 32
# Final iteration/time
msrmnt = np.array([100])
# For training or testing dataset
train = False
# generate animations instead
animate = True
# number of samples 
samples = 1
for iter in range(samples):
    for temp in temp_dist:
        for duration in msrmnt:
            start = time.time()
            rm = Ising(temp = temp, n = N, msrmnt = duration, animate=animate, train = train, verbose = False)
            rm.simulate()
            end = time.time()
            print(time_parse(end - start))

animation/IsingXY_0.3212437065906615_0.gif
Time Elapsed: 9.18 seconds
animation/IsingXY_0.4466704953022078_0.gif
Time Elapsed: 9.27 seconds
animation/IsingXY_1.884388711458901_0.gif
Time Elapsed: 9.26 seconds
animation/IsingXY_0.13301498710073312_0.gif
Time Elapsed: 9.24 seconds
animation/IsingXY_0.37968715369510364_0.gif
Time Elapsed: 9.16 seconds
animation/IsingXY_1.898157999945422_0.gif
Time Elapsed: 9.12 seconds
animation/IsingXY_0.010535038293034038_0.gif
Time Elapsed: 9.19 seconds
animation/IsingXY_1.5487474565859805_0.gif
Time Elapsed: 9.21 seconds
animation/IsingXY_1.7711811336893624_0.gif
Time Elapsed: 9.21 seconds
animation/IsingXY_0.7539702928169689_0.gif
Time Elapsed: 9.44 seconds
